In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import time
import seaborn as sb

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4
epochs = 5
sb.set_style('darkgrid')

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print("Train dataset :", len(testloader))

print("Test dataset :", len(testloader))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels ---Number = batch_size
print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
input_shape = (3, 32, 32)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
#Display details about the 2 Layered neural system
summary(net, input_shape)

In [ ]:
#My contribution---- 4 layered NN

class Net4CL(nn.Module):
  def __init__(self):
    super(Net4CL, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
    self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
    self.pool = nn.MaxPool2d(2, 2)
    self.dropout = nn.Dropout(0.1)
    self.fc1 = nn.Linear(128 * 2 * 2, 512)
    self.fc2 = nn.Linear(512, 256)
    self.fc3 = nn.Linear(256, 10)
    
  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = self.pool(F.relu(self.conv3(x)))
    x = self.pool(F.relu(self.conv4(x)))
    x = x.view(-1, 128 * 2 * 2)
    x = self.dropout(x)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

net4cl = Net4CL()
summary(net4cl, input_shape)

In [ ]:
def graphModel(model, trainloader, optimizer, criterion):
  lossList = []
  print("/*Training...this process might take several minutes to complete depending on the train/test size........do not execute next cell until finished")
  for epoch in range(epochs):
    training_loss = 0.0
    for i, data in enumerate(trainloader, 0):
      inputs, labels = data
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      #print statistics
      training_loss += loss.item()
      if i % 5000 == 4999: # print every 5000 mini-batches
        print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, training_loss / 5000))
        training_loss = 0.0
    
    # calculate average losses
    training_loss = training_loss/len(trainloader)
    lossList.append(training_loss)

  print("Training Finished.")
  return lossList

def modelLayer(model):
  if model == "net2cl": return Net()
  elif model == "net4cl": return Net4CL()

def plot_loss_per_epoch(history):
  plt.plot(np.arange(0, epochs), history, label='training_loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend(loc='center right')
  plt.show()

In [ ]:
import torch.optim as optim

#Cross-Entropy Loss
criterion = nn.CrossEntropyLoss()

net = "net2cl"  ###---------------------------- 'net2cl' or 'net4cl' 
model = modelLayer(net)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
lossList= graphModel(model, trainloader, optimizer, criterion)
plot_loss_per_epoch(lossList)

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))

In [ ]:
outputs = model(images)
_, predicted = torch.max(outputs, 1)
print('Predicted: ', ' '.join('%5s' %classes[predicted[j]] for j in range(batch_size)))

In [ ]:
def accuracy(net, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = (100*correct / total)
    print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))
    return accuracy 

In [ ]:
# total accuracy
nn2accuracy = []
#appending accuracy of 2 layered NN 
nn2accuracy.append(accuracy(model, testloader))

In [ ]:
# SGD optimizer & Cross Entropy Loss
net = "net4cl"  ###---------------------------- 'net2cl' or 'net4cl' 
model = modelLayer(net)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
lossList= graphModel(model, trainloader, optimizer, criterion)
plot_loss_per_epoch(lossList)

In [ ]:
#appending accuracy of 4 layered NN 
nn2accuracy.append(accuracy(model, testloader))

In [ ]:
def plot_classes_accuracy(classes, accuracy):
  plt.bar(classes, accuracy)
  plt.xlabel("Class")
  plt.ylabel("Accuracy")
  plt.show()

#Comparing performance of 2 layered and 4 layered CNN
nnList = ['Net2CL','Net4CL']
plot_classes_accuracy(nnList, nn2accuracy)